## Background

In this article, we will use [softmax](https://en.wikipedia.org/wiki/Softmax_function) classifier to build a simple image classification neural network with an accuracy of 32%. In a Softmax classifier, binary logic is generalized and regressed to multiple logic. Softmax classifier will output the probability of the corresponding category.

We will first define a softmax classifier, then use the training set of [CIFAR10](https://www.cs.toronto.edu/~kriz/cifar.html) to train the neural network, and finally use the test set to verify the accuracy of the neural network.

Let’s get started.

## Import dependencies

Like the previous course [GettingStarted](https://thoughtworksinc.github.io/DeepLearning.scala/demo/GettingStarted.html), we need to introduce each class of DeepLearning.scala.

In [19]:
import $ivy.`org.nd4j::nd4s:0.8.0`
// import $ivy.`org.nd4j:nd4j-api:0.8.0`
// import $ivy.`org.nd4j:nd4j-cuda-8.0-platform:0.8.0`
import $ivy.`org.nd4j:nd4j-native-platform:0.8.0`
import $ivy.`com.chuusai::shapeless:2.3.2`
import $ivy.`org.rauschig:jarchivelib:0.5.0`
import $ivy.`com.thoughtworks.deeplearning::plugins-builtins:2.0.1`
import $ivy.`org.plotly-scala::plotly-jupyter-scala:0.3.2`
import $ivy.`com.thoughtworks.each::each:3.3.1`
import $plugin.$ivy.`org.scalamacros:paradise_2.11.11:2.1.0`

import scala.concurrent.ExecutionContext.Implicits.global
import org.nd4j.linalg.api.ndarray.INDArray
import org.nd4j.linalg.factory.Nd4j
import com.thoughtworks.deeplearning.DeepLearning
import com.thoughtworks.deeplearning.plugins._
import com.thoughtworks.feature.Factory
import plotly._
import plotly.element._
import plotly.layout._
import plotly.JupyterScala._
plotly.JupyterScala.init()

import com.thoughtworks.future._
import scala.concurrent.Await
import scala.concurrent.duration.Duration
import com.thoughtworks.each.Monadic._
import scalaz.std.stream._

import $ivy.$                     
// import $ivy.`org.nd4j:nd4j-api:0.8.0`
// import $ivy.`org.nd4j:nd4j-cuda-8.0-platform:0.8.0`

import $ivy.$                                    

import $ivy.$                             

import $ivy.$                               

import $ivy.$                                                      

import $ivy.$                                             

import $ivy.$                                  

import $plugin.$                                            


import scala.concurrent.ExecutionContext.Implicits.global

import org.nd4j.linalg.api.ndarray.INDArray

import org.nd4j.linalg.factory.Nd4j

import com.thoughtworks.deeplearning.DeepLearning

import com.thoughtworks.deeplearning.plugins._

import com.thoughtworks.feature.Factory

import plotly._

import plotly.element._

import plotly.layout._

import plotly.JupyterScala._

import com.thoughtworks.future._

import scala.concurrent.Await

import scala.concurrent.duration.Duration

i

To reduce the line numbers outputted by `jupyter-scala` and to make sure that the page output will not be too long, we need to set `pprintConfig`.

In [20]:
pprintConfig() = pprintConfig().copy(height = 2)

## Build your own neural network.

### Set learning rate

Learning rate need to be set for the full connection layer. Learning rate visually describes the change rate of `weight`. A too-low learning rate will result in slow decrease of `loss`, which will require longer time for training; A too-high learning rate will result in rapid decrease of `loss` at first while fluctuation around the lowest point afterward.

In [22]:
val INDArrayLearningRatePluginUrl = "https://gist.githubusercontent.com/TerrorJack/118487016d7973d67feb489449dee156/raw/778bb1b68a664c752b0945111220326731310214/INDArrayLearningRate.sc"
interp.load(scala.io.Source.fromURL(new java.net.URL(INDArrayLearningRatePluginUrl)).mkString)

INDArrayLearningRatePluginUrl: String = "https://gist.githubusercontent.com/TerrorJack/118487016d7973d67feb489449dee156/raw/778bb1b68a664c752b0945111220326731310214/INDArrayLearningRate.sc"

In [23]:
interp.load(scala.io.Source.fromURL(new java.net.URL("https://gist.github.com/Atry/15b7d9a4c63d95ad3d67e94bf20b4f69/raw/59f7ee4dff0dde3753f560633574265e950edc93/CNN.sc")).mkString)

Main1.sc:62: value forward is not a member of type parameter Input
      val inputShape = input.forward.each.data.shape
                             ^Main1.sc:67: value forward is not a member of type parameter Weight
      val numberOfKernels = weight.forward.each.data.shape.head
                                   ^Main1.sc:69: value permute is not a member of type parameter Out
      val permutedCol = col.permute(0, 4, 5, 1, 2, 3)
                            ^Main1.sc:72: value reshape is not a member of type parameter Weight
      val reshapedWeight = weight.reshape(numberOfKernels, depthKernelKernel)
                                  ^

: 

In [25]:
// `interp.load` is a workaround for https://github.com/lihaoyi/Ammonite/issues/649 and https://github.com/scala/bug/issues/10390
interp.load("""
  val hyperparameters = Factory[Builtins with CNNs with INDArrayLearningRate].newInstance(learningRate = 0.1)
""")

### Write softmax

To use `softmax` classifier (softmax classifier is a neural network combined by `softmax` and a full connection), we first need to write softmax function, formula: ![](https://www.zhihu.com/equation?tex=f_j%28z%29%3D%5Cfrac%7Be%5E%7Bz_j%7D%7D%7B%5Csum_ke%5E%7Bz_k%7D%7D)

In [26]:
import hyperparameters.implicits._

import hyperparameters.implicits._

In [27]:
import hyperparameters.INDArrayLayer

def softmax(scores: INDArrayLayer): INDArrayLayer = {
  val expScores = hyperparameters.exp(scores)
  expScores / expScores.sum(1)
}

import hyperparameters.INDArrayLayer


defined function softmax

In [28]:
val fileHandler = new java.util.logging.FileHandler("CNN%g.log")
hyperparameters.logger.addHandler(fileHandler)

fileHandler: java.util.logging.FileHandler = java.util.logging.FileHandler@7a27354b

### Compose your  neural network

Define a full connection layer and [initialize Weight](https://github.com/ThoughtWorksInc/DeepLearning.scala/wiki/Getting-Started#231--weight-intialization), `Weight` shall be a two-dimension `INDArray` of `NumberOfPixels × NumberOfClasses`. `scores` is the score of each image corresponding to each category, representing the feasible probability of each category corresponding to each image.

In [29]:
//10 label of CIFAR10 images(airplane,automobile,bird,cat,deer,dog,frog,horse,ship,truck)
val NumberOfClasses: Int = 10
val NumberOfPixels: Int = 3072

NumberOfClasses: Int = 10
NumberOfPixels: Int = 3072

In [30]:
import hyperparameters.INDArrayWeight

val weight1 = {
    import org.nd4s.Implicits._
    INDArrayWeight(Nd4j.randn(Array(16, 3, 3, 3)) / math.sqrt(3 * 3 * 3 / 2))
}
val bias1 = {
    hyperparameters.INDArrayWeight(Nd4j.zeros(16))
}

val weight2 = {
    import org.nd4s.Implicits._
    INDArrayWeight(Nd4j.randn(Array(18, 16, 3, 3)) / math.sqrt(16 * 3 * 3 / 2))
}
val bias2 = {
    hyperparameters.INDArrayWeight(Nd4j.zeros(18))
}

val weight3 = {
    import org.nd4s.Implicits._
    INDArrayWeight(Nd4j.randn(Array(20, 18, 3, 3)) / math.sqrt(18 * 3 * 3 / 2))
}
val bias3 = {
    hyperparameters.INDArrayWeight(Nd4j.zeros(20))
}

val weight4 = {
    import org.nd4s.Implicits._
    INDArrayWeight(Nd4j.randn(Array(22, 20, 3, 3)) / math.sqrt(20 * 3 * 3 / 2))
}
val bias4 = {
    hyperparameters.INDArrayWeight(Nd4j.zeros(22))
}

val weight5 = {
    import org.nd4s.Implicits._
    INDArrayWeight(Nd4j.randn(Array(24, 22, 3, 3)) / math.sqrt(22 * 3 * 3 / 2))
}
val bias5 = {
    hyperparameters.INDArrayWeight(Nd4j.zeros(24))
}


val weight6 = {
    import org.nd4s.Implicits._
    INDArrayWeight(Nd4j.randn(Array(24, 10)) / math.sqrt(24 / 2))
}
val bias6 = {
    hyperparameters.INDArrayWeight(Nd4j.zeros(10))
}

def myNeuralNetwork(input: INDArray): INDArrayLayer = {
    import hyperparameters.max
    import hyperparameters.maxPool
    import hyperparameters.conv2d
    val layer1 = maxPool(max(conv2d(input.reshape(input.shape()(0), 3, 32, 32), weight1, bias1, (3, 3), (1, 1), (1, 1)), 0.0), (2, 2))
    val layer2 = maxPool(max(conv2d(layer1, weight2, bias2, (3, 3), (1, 1), (1, 1)), 0.0), (2, 2))
    val layer3 = maxPool(max(conv2d(layer2, weight3, bias3, (3, 3), (1, 1), (1, 1)), 0.0), (2, 2))
    val layer4 = maxPool(max(conv2d(layer3, weight4, bias4, (3, 3), (1, 1), (1, 1)), 0.0), (2, 2))
    val layer5 = maxPool(max(conv2d(layer4, weight5, bias5, (3, 3), (1, 1), (1, 1)), 0.0), (2, 2))

    val layer6 = layer5.reshape(input.shape()(0), 24) dot weight6 + bias6
    softmax(layer6)
}

import hyperparameters.INDArrayWeight


weight1: Object with hyperparameters.INDArrayWeightApi with hyperparameters.WeightApi with hyperparameters.WeightApi = Weight[fullName=$sess.cmd29Wrapper.Helper.weight1]
bias1: Object with hyperparameters.INDArrayWeightApi with hyperparameters.WeightApi with hyperparameters.WeightApi = Weight[fullName=$sess.cmd29Wrapper.Helper.bias1]
weight2: Object with hyperparameters.INDArrayWeightApi with hyperparameters.WeightApi with hyperparameters.WeightApi = Weight[fullName=$sess.cmd29Wrapper.Helper.weight2]
bias2: Object with hyperparameters.INDArrayWeightApi with hyperparameters.WeightApi with hyperparameters.WeightApi = Weight[fullName=$sess.cmd29Wrapper.Helper.bias2]
weight3: Object with hyperparameters.INDArrayWeightApi with hyperparameters.WeightApi with hyperparameters.WeightApi = Weight[fullName=$sess.cmd29Wrapper.Helper.weight3]
bias3: Object with hyperparameters.INDArrayWeightApi with hyperparameters.WeightApi with hyperparameters.WeightApi = W

### Create LossFunction

To learn about the prediction result of the neural network, we need to write the loss function `lossFunction`. We use [cross-entropy loss](https://en.wikipedia.org/wiki/Cross_entropy) to make comparison between this result and the actual result before return the score. Formula:
![](https://zhihu.com/equation?tex=%5Cdisplaystyle+H%28p%2Cq%29%3D-%5Csum_xp%28x%29+logq%28x%29)

In [31]:
import hyperparameters.DoubleLayer

def lossFunction(input: INDArray, expectOutput: INDArray): DoubleLayer = {
    val probabilities = myNeuralNetwork(input)
    -(hyperparameters.log(probabilities) * expectOutput).mean
}

import hyperparameters.DoubleLayer


defined function lossFunction

## Prepare data

### Read data

To read the images and corresponding label information for test data from CIFAR10 database and process them, we need [`import $file.ReadCIFAR10ToNDArray`](https://github.com/ThoughtWorksInc/DeepLearning.scala-website/blob/master/ipynbs/ReadCIFAR10ToNDArray.sc). This is a script file containing the read and processed CIFAR10 data, provided in this course.

In [32]:
// import $url.{`https://raw.githubusercontent.com/ThoughtWorksInc/DeepLearning.scala-website/v1.0.0-doc/ipynbs/ReadCIFAR10ToNDArray.sc` => ReadCIFAR10ToNDArray}

// val trainNDArray = ReadCIFAR10ToNDArray.readFromResource("/cifar-10-batches-bin/data_batch_1.bin", 1000)

// val testNDArray = ReadCIFAR10ToNDArray.readFromResource("/cifar-10-batches-bin/test_batch.bin", 100)

import $ivy.`com.thoughtworks.deeplearning.etl::cifar10:1.0.1`
import com.thoughtworks.deeplearning.etl.Cifar10
import com.thoughtworks.future._
val cifar10 = Cifar10.load().blockingAwait

import $ivy.$                                                 

import com.thoughtworks.deeplearning.etl.Cifar10

import com.thoughtworks.future._

cifar10: Cifar10 = Cifar10(
  Vector(
...

### Process data

Before passing data to the softmax classifier, we first process label data with ([one hot encoding](https://en.wikipedia.org/wiki/One-hot)): transform INDArray of `NumberOfPixels × 1` into INDArray of `NumberOfPixels × NumberOfClasses`. The value of correct classification corresponding to each line is 1, and the values of other columns are 0. The reason for differentiating the training set and test set is to make it clear that whether the network is over trained which leads to [overfitting](https://en.wikipedia.org/wiki/Overfitting). While processing label data, we used [Utils](https://github.com/ThoughtWorksInc/DeepLearning.scala-website/blob/master/ipynbs/Utils.sc), which is also provided in this course.

In [32]:
// val trainData = trainNDArray.head
// val testData = testNDArray.head

// val trainExpectResult = trainNDArray.tail.head
// val testExpectResult = testNDArray.tail.head

// import $url.{`https://raw.githubusercontent.com/ThoughtWorksInc/DeepLearning.scala-website/v1.0.0-doc/ipynbs/Utils.sc` => Utils}

// val vectorizedTrainExpectResult = Utils.makeVectorized(trainExpectResult, NumberOfClasses)
// val vectorizedTestExpectResult = Utils.makeVectorized(testExpectResult, NumberOfClasses)

## Train your neural network

To observe the training process of the neural network, we need to output `loss`; while training the neural network, the `loss` shall be decreasing.

In [33]:
// var lossSeq: IndexedSeq[Double] = IndexedSeq.empty

// @monadic[Future]
// val trainTask: Future[Unit] = {
//   val lossStream = for (_ <- (1 to 2000).toStream) yield {
//     val loss = lossFunction(trainData, vectorizedTrainExpectResult).train.each
//     kernel.publish.markdown(s"loss: $loss")
//     loss
//   }
//   lossSeq = IndexedSeq.concat(lossStream)
// }

import scalaz.std.anyVal._
import scalaz.syntax.all._

class Trainer(batchSize: Int, numberOfEpoches: Int = 5) {
    @volatile
    private var isShuttingDown: Boolean = false

    private val lossBuffer = scala.collection.mutable.Buffer.empty[Double]
        
    def poltLoss(): Unit = Seq(Scatter(lossBuffer.indices, lossBuffer)).plot(title = "loss by time")
    
    def interrupt(): Unit = isShuttingDown = true

    def startTrain(): Unit = {

        @monadic[Future]
        def trainTask: Future[Unit] = {
            isShuttingDown = false
            var epoch = 0
            while (epoch < numberOfEpoches && !isShuttingDown) {
                val iterator = cifar10.epoch(batchSize).zipWithIndex
                while (iterator.hasNext && !isShuttingDown) {
                    val (Cifar10.Batch(labels, batch), i) = iterator.next()
                    val loss = lossFunction(batch, labels).train.each
                    lossBuffer += loss
                    hyperparameters.logger.info(s"epoch=$epoch iteration=$i batchSize=$batchSize loss=$loss")
                }
                epoch += 1
            }
            hyperparameters.logger.info("Done")
        }

        trainTask.onComplete { tryUnit: scala.util.Try[Unit] => tryUnit.get }

    }
}




import scalaz.std.anyVal._

import scalaz.syntax.all._


defined class Trainer

In [34]:
val trainer = new Trainer(batchSize = 32, numberOfEpoches = 300)
trainer.startTrain()

trainer: Trainer = $sess.cmd32Wrapper$Helper$Trainer@8c45e71

In [35]:
//
trainer.interrupt()

## Predict  your Neural Network

We use the processed test data to verify the prediction result of the neural network and compute the accuracy. The accuracy shall be about 32%.

In [18]:
// val predictResult = Await.result(myNeuralNetwork(testData).predict.toScalaFuture, Duration.Inf)

// /**
//     * 计算准确率
//     * @param score 预测结果
//     * @param testExpectLabel 期望结果
//     * @return 准确率
//     */
// def getAccuracy(score: INDArray, testExpectLabel: INDArray): Double = {
//     val scoreIndex = findMaxItemIndex(score)
//     if (testExpectLabel.shape().toSeq.last == 1) { //not vectorized
//         val numberOfCorrectPrediction = (0 until scoreIndex.shape()(0)).count { row =>
//             scoreIndex.getDouble(row, 0) == testExpectLabel.getDouble(row, 0)
//         }
//         (numberOfCorrectPrediction / score.shape()(0)) * 100
//     } else if (testExpectLabel.shape().toSeq.last == 10) { //vectorized
//           val expectResultIndex = findMaxItemIndex(testExpectLabel)
//           val accINDArray = scoreIndex.eq(expectResultIndex)
//           (accINDArray.sumT / score.shape()(0)) * 100
//     } else
//       throw new IllegalArgumentException("Unacceptable testExpectLabel")
// }

// println("The accuracy is " + getAccuracy(predictResult,testExpectResult) + "%")

In [36]:
trainer.poltLoss()

## Summary

We have learned the follows in this article:

* Prepare and process CIFAR10 data
* Write softmax classifier
* Use the prediction image of the neural network written by softmax classifier to match with the probability of each category.